# Speech + Phonetic AAI Model

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
import matplotlib.pyplot as plt
import os
from os.path import join
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers
from tensorflow.keras import callbacks
from time import time
from datetime import datetime
import shutil 
import argparse
from functools import partial
import pickle

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Reshape, GRU, InputLayer, AlphaDropout, Activation, BatchNormalization, Dropout, Flatten, Dense, Bidirectional, LSTM, Conv1D, SpatialDropout1D, Concatenate

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import OneHotEncoder

## Utility Functions

In [2]:
#Function for preprocessing data
def delete_params(params):
    '''
    This function remove JX, WC, TRX, TRY, and MS1,2,3 paramter
    '''
    DEL_PARAMS_LIST = [2,8,15,16,21,22,23]
    return np.delete(params,DEL_PARAMS_LIST , axis=1)

## Load Data 

In [3]:
def prep_data():

    # load data from preprocess pipeline
    dataset = np.load(join('../data/d_dataset_p1/prep_data_13','training_subsets.npz'))
    X_train =dataset['X_train']
    y_train= dataset['y_train']
    z_train= dataset['z_train']
    X_val = dataset['X_val']
    y_val = dataset['y_val']
    z_val = dataset['z_val']
    X_test = dataset['X_test']
    y_test = dataset['y_test']
    z_test = dataset['z_test']
    
    y_train = delete_params(y_train)
    y_val = delete_params(y_val)
    y_test = delete_params(y_test)

    print('Train features and labels %s %s'%(str(X_train.shape),str(y_train.shape)))
    print('Validating features and labels %s %s'%(str(X_val.shape),str(y_val.shape)))
    print('Test features and labels %s %s'%(str(X_test.shape),str(y_test.shape)))

    return X_train, X_val, X_test, y_train, y_val, y_test, z_train, z_val, z_test

In [4]:
X_train, X_val, X_test, y_train, y_val, y_test, z_train, z_val, z_test = prep_data()

Train features and labels (98746, 24, 39) (98746, 17)
Validating features and labels (12094, 24, 39) (12094, 17)
Test features and labels (12092, 24, 39) (12092, 17)


In [5]:
print(set(z_test))

{'2', 'U', '@', 'O', 'o', 'A', 'y', 'E', 'i', '7', 'a', 'Q', 'V', 'e', '&', 'M', 'u', 'E:', '9'}


In [6]:
print('Train features and labels {} {} {}'.format(str(X_train.shape),str(y_train.shape),str(z_train.shape)))
print('Validating features and labels {} {} {}'.format(str(X_val.shape),str(y_val.shape),str(z_val.shape)))
print('Test features and labels {} {} {}'.format(str(X_test.shape),str(y_test.shape),str(z_test.shape)))

Train features and labels (98746, 24, 39) (98746, 17) (98746,)
Validating features and labels (12094, 24, 39) (12094, 17) (12094,)
Test features and labels (12092, 24, 39) (12092, 17) (12092,)


### One-hot label 

In [7]:
z_train_oe=z_train.reshape(-1,1)
z_val_oe=z_val.reshape(-1,1)
z_test_oe=z_test.reshape(-1,1)

In [8]:
enc = OneHotEncoder(handle_unknown = 'ignore')
enc.fit(z_train_oe)

z_train_oe = enc.transform(z_train_oe).toarray()
z_test_oe = enc.transform(z_test_oe).toarray()
z_val_oe = enc.transform(z_val_oe).toarray()

In [9]:
z_train_oe.shape

(98746, 19)

In [10]:
z_val_oe.shape

(12094, 19)

In [11]:
z_train_oe[1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.])

## Defined Model 

In [12]:
N_OUTPUTS = 17

pLSTM = partial(LSTM,
    kernel_initializer='he_uniform',
    return_sequences=True)

pDense = partial(Dense,
    kernel_initializer='he_normal',
    activation='elu')

def init_bilstm_pho(unit=128, bi_layer_num=3, drop_rate=0.2):

    def bilstm(input_shape_1,input_shape_2):
        
        input_x = keras.Input(shape=(input_shape_1,input_shape_2))
        # feature extraction layers
        x = input_x
        for i in range(bi_layer_num-1):
            x = Bidirectional(pLSTM(unit))(x)
            if drop_rate: 
                x = SpatialDropout1D(rate=drop_rate)(x)
        # output layers
        x = Bidirectional(pLSTM(unit*2, return_sequences=False))(x)
        if drop_rate: 
            x = Dropout(rate=drop_rate)(x)
        x = pDense(1024, activation='relu')(x)
        outputs_param = pDense(N_OUTPUTS, activation='linear')(x)
        
        model = keras.Model(inputs=input_x, outputs=outputs_param)
        # model.summary()
        return model
    return bilstm

In [13]:
model = init_bilstm()(24, 39)

NameError: name 'init_bilstm' is not defined

In [ ]:
model.summary()

## Metric Function

In [ ]:
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))

def R2(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred), axis = 0) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true, axis = 0)), axis = 0) 
    return K.mean(1 - (SS_res/SS_tot), axis=0)

## Compile Model 

In [ ]:
model.compile(optimizer='adam',loss='mse',metrics=[rmse, R2])

## Train Model 

In [ ]:
PATIENCE = 10
BATCH_SIZE = 128
EPOCHS = 30

In [ ]:
early = callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=PATIENCE, verbose=1, mode='min', baseline=None, restore_best_weights=False)
callback_list = [early]

In [ ]:
history = model.fit(X_train,y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(X_val,y_val),
    callbacks=callback_list)

In [ ]:
model.save('model_baseline.hdf5')

## Evaluation 

In [ ]:
model.evaluate(X_test,y_test,verbose=0)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
def compute_rmse(actual,pred, axis=1):
    # Compute RMSE by row (axis=1) result in rmse of each data
    # Compute RMSE by column (axis=0) result in rmse of each label
    return np.sqrt((np.square(actual - pred)).mean(axis=axis))

In [ ]:
err = compute_rmse(y_test,y_pred, axis=0)

In [ ]:
print(err)

## Defined Model + Phonetic

In [ ]:
N_OUTPUTS = 17

pLSTM = partial(LSTM,
    kernel_initializer='he_uniform',
    return_sequences=True)

pDense = partial(Dense,
    kernel_initializer='he_normal',
    activation='elu')

def init_bilstm_pho(unit=128, bi_layer_num=3, drop_rate=0.2):

    def bilstm(input_shape_1,input_shape_2):
        
        input_x = keras.Input(shape=(input_shape_1,input_shape_2))
        # feature extraction layers
        x = input_x
        z = input_x
        for i in range(bi_layer_num-1):
            x = Bidirectional(pLSTM(unit))(x)
            if drop_rate: 
                x = SpatialDropout1D(rate=drop_rate)(x)
        # output layers
        x = Bidirectional(pLSTM(unit*2, return_sequences=False))(x)
        if drop_rate: 
            x = Dropout(rate=drop_rate)(x)
                
        for i in range(bi_layer_num-2):
            z = Bidirectional(pLSTM(unit))(z)
            if drop_rate: 
                z = SpatialDropout1D(rate=drop_rate)(z)
        z = Bidirectional(pLSTM(unit*2, return_sequences=False))(z)
        if drop_rate: 
            z = Dropout(rate=drop_rate)(z)
        z = pDense(100, activation='relu')(z)
        outputs_pho = pDense(19, activation='softmax', name='pho')(x)
        
        x = Concatenate()([x,z])
        x = pDense(512, activation='relu')(x)
        outputs_param = pDense(N_OUTPUTS, activation='linear', name='param')(x)
        
        model = keras.Model(inputs=input_x, outputs=[outputs_param, outputs_pho])
        # model.summary()
        return model
    return bilstm

In [ ]:
model_pho = init_bilstm_pho()(24, 39)

In [ ]:
model_pho.summary()

In [ ]:
losses = {
    "param": "mse",
    "pho": tf.keras.losses.CategoricalCrossentropy(from_logits=True),
}
lossWeights = {"param": 1.0, "pho": 1.0}

In [ ]:
metrics = {
    "param": [rmse, R2],
    "pho": "accuracy",
}

## Compile Model 

In [ ]:
model_pho.compile(optimizer='adam',loss=losses, loss_weights=lossWeights, metrics=metrics)

## Train Model 

In [ ]:
PATIENCE = 10
BATCH_SIZE = 128
EPOCHS = 30

In [ ]:
early = callbacks.EarlyStopping(monitor='param_loss', min_delta=0, patience=PATIENCE, verbose=1, mode='min', baseline=None, restore_best_weights=False)
callback_list = [early]

In [ ]:
history = model_pho.fit(X_train,{"param": y_train, "pho": z_train_oe},
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    verbose=2,
    validation_data=(X_val,{"param": y_val, "pho": z_val_oe}),
    callbacks=callback_list)

In [ ]:
model_pho.save('model_baseline_pho.hdf5')

## Evaluation

In [ ]:
def evaluate_model(y_pred, label, model):
    y_pred = enc.inverse_transform(y_pred)
    label = enc.inverse_transform(label)
    print('Confusion Matrix')
    print(confusion_matrix(label, y_pred))
    print('Classification Report')
    print(classification_report(label, y_pred))

In [ ]:
y_pred, y_pho_pred = model.predict(X_test)

In [ ]:
err = compute_rmse(y_test,y_pred, axis=0)
print(err)

In [ ]:
evaluate_model(y_pho_pred, y_test, model_pho)